In [41]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Model Evaluations
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
import time
import sys
import pickle
 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("symbol_features.csv")
df

,modulation_type,symbol,label,magnitude,phase,real,imag
0,bpsk,(1+0j),1,1.000000,0.000000,1.000000,0.000000
1,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
2,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
3,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
4,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
...,...,...,...,...,...,...,...
795,256apsk,(-0.2888089479520107+0.8888625445008081j),0,0.934605,1.884956,-0.288809,0.888863
796,256apsk,(-0.28880894795201084-0.888862544500808j),0,0.934605,-1.884956,-0.288809,-0.888863
797,256apsk,(0.6776553913499502-0.9516338422362578j),1,1.168257,-0.951998,0.677655,-0.951634
798,256apsk,(1.0845720904002638-0.4341971670103332j),0,1.168257,-0.380799,1.084572,-0.434197


In [3]:

feature_cols = ['magnitude', 'phase', 'real', 'imag','label']
target_col = 'modulation_type'

X_train, X_test, y_train, y_test = train_test_split(df[feature_cols], df[target_col], test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.3f}')

# # Print the classification report
# print(classification_report(y_test, y_pred))

# # Print the confusion matrix
# print(confusion_matrix(y_test, y_pred))

In [4]:
clf.score(X_test,y_test)

0.925

In [5]:
y_pred = clf.predict(X_test)
y_pred

array(['128apsk', '128apsk', 'bpsk', '64apsk', 'bpsk', '128apsk',
       '16apsk', '32apsk', '256apsk', '32apsk', 'bpsk', '8apsk',
       '128apsk', 'bpsk', '16apsk', '16apsk', '8apsk', '16apsk',
       '128apsk', '64apsk', '128apsk', '64apsk', '128apsk', '256apsk',
       '16apsk', 'bpsk', '8apsk', '128apsk', '256apsk', '256apsk',
       '8apsk', 'bpsk', '64apsk', '16apsk', 'bpsk', 'bpsk', '128apsk',
       '16apsk', '256apsk', '256apsk', 'qpsk', '8apsk', 'qpsk', '16apsk',
       '64apsk', '128apsk', '32apsk', '64apsk', '128apsk', '64apsk',
       '8apsk', 'qpsk', '128apsk', '8apsk', '16apsk', '16apsk', '32apsk',
       '16apsk', 'qpsk', 'qpsk', '32apsk', '128apsk', '16apsk', '16apsk',
       '8apsk', '32apsk', 'bpsk', '256apsk', '16apsk', 'bpsk', '64apsk',
       'qpsk', '256apsk', 'bpsk', 'bpsk', 'qpsk', '16apsk', '32apsk',
       '256apsk', '64apsk', '32apsk', 'qpsk', 'bpsk', 'bpsk', '64apsk',
       '16apsk', '128apsk', '8apsk', '16apsk', 'bpsk', '32apsk',
       '256apsk', '8apsk

In [6]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     128apsk       0.86      0.82      0.84        22
      16apsk       0.92      1.00      0.96        23
     256apsk       0.90      0.78      0.84        23
      32apsk       0.81      1.00      0.90        13
      64apsk       0.95      0.86      0.90        22
       8apsk       1.00      1.00      1.00        20
        bpsk       0.96      1.00      0.98        24
        qpsk       1.00      1.00      1.00        13

    accuracy                           0.93       160
   macro avg       0.92      0.93      0.93       160
weighted avg       0.93      0.93      0.92       160



In [7]:
df_noise=pd.read_csv("symbol_features_noise.csv")
df_noise

,modulation_type,symbol,label,magnitude,phase,real,imag
0,bpsk,(1+0j),1,1.000000,0.000000,1.000000,0.000000
1,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
2,bpsk,(1+0j),1,1.000000,0.000000,1.000000,0.000000
3,bpsk,(1+0j),1,1.000000,0.000000,1.000000,0.000000
4,bpsk,(-1+0j),0,1.000000,3.141593,-1.000000,0.000000
...,...,...,...,...,...,...,...
795,256apsk,(0.1943153864837051-0.9141820179861813j),1,0.934605,-1.361357,0.194315,-0.914182
796,256apsk,(-0.6316072762644357-0.9828001056597511j),1,1.168257,-2.141995,-0.631607,-0.982800
797,256apsk,(0.711853812279336+0.16247598737250918j),0,0.730160,0.224399,0.711854,0.162476
798,256apsk,(0.8455071644399965-0.8061894529176338j),1,1.168257,-0.761598,0.845507,-0.806189


In [8]:
feature_cols = ['magnitude', 'phase', 'real', 'imag','label']
target_col = 'modulation_type'

X_train, X_test, y_train, y_test = train_test_split(df_noise[feature_cols], df_noise[target_col], test_size=0.2, random_state=42)

clf2 = RandomForestClassifier(n_estimators=100, random_state=42)

clf2.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [9]:
clf2.score(X_test,y_test)

0.93125

In [10]:
y_pred = clf.predict(X_test)
y_pred

array(['128apsk', '128apsk', 'bpsk', '128apsk', 'bpsk', '64apsk',
       '16apsk', '32apsk', '256apsk', '32apsk', 'bpsk', '8apsk',
       '128apsk', 'bpsk', '16apsk', '16apsk', '8apsk', '16apsk',
       '128apsk', '64apsk', '128apsk', '128apsk', '128apsk', '256apsk',
       '16apsk', 'bpsk', '8apsk', '128apsk', '256apsk', '64apsk', '8apsk',
       'bpsk', '64apsk', '16apsk', 'bpsk', 'bpsk', '128apsk', '16apsk',
       '256apsk', '256apsk', 'qpsk', '8apsk', 'qpsk', '16apsk', '64apsk',
       '64apsk', '32apsk', '64apsk', '256apsk', '64apsk', '8apsk', 'qpsk',
       '64apsk', '8apsk', '16apsk', '16apsk', '32apsk', '16apsk', 'qpsk',
       'qpsk', '32apsk', '256apsk', '128apsk', '16apsk', '8apsk',
       '32apsk', 'bpsk', '256apsk', '16apsk', 'bpsk', '64apsk', 'qpsk',
       '256apsk', 'bpsk', 'bpsk', 'qpsk', '16apsk', '32apsk', '64apsk',
       '64apsk', '32apsk', 'qpsk', 'bpsk', 'bpsk', '64apsk', '32apsk',
       '256apsk', '8apsk', '16apsk', 'bpsk', '128apsk', '32apsk', '8apsk',
      

In [11]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

     128apsk       0.86      0.90      0.88        21
      16apsk       1.00      0.96      0.98        24
     256apsk       0.83      0.90      0.86        21
      32apsk       1.00      0.87      0.93        15
      64apsk       0.86      0.86      0.86        22
       8apsk       1.00      1.00      1.00        20
        bpsk       1.00      1.00      1.00        24
        qpsk       1.00      1.00      1.00        13

    accuracy                           0.94       160
   macro avg       0.94      0.94      0.94       160
weighted avg       0.94      0.94      0.94       160



In [51]:
def weighted_score(model, X_test, y_test, weight_for_accuracy=0.7, weight_for_latency = 0.2, weight_for_model_size=0.1):
    try:
        accuracy = model.score(X_test, y_test)        
        start_time = time.time()
        _ = model.predict(X_test)
        end_time = time.time()
        latency = (end_time - start_time) / len(X_test)

        model_pickle = pickle.dumps(clf)
        model_size = len(model_pickle)

        max_latency = 0.01
        max_model_size = 1e6

        normalized_latency = latency/max_latency
        normalized_size = model_size/max_model_size

        score = (weight_for_accuracy * accuracy) - (weight_for_latency * normalized_latency) - (weight_for_model_size * normalized_size)
        
        result_string = (
            f"Accuracy: {accuracy:.3f}\n"
            f"Normalized Latency: {normalized_latency:.3f} and Latency: {latency}\n"
            f"Normalized Size: {normalized_size:.3f} and Size: {model_size} in Bytes\n"
            f"Final Power-Efficient Score: {score:.3f}")

        return result_string

    
    except Exception as e:
        print(f"Error calculating power-efficient score: {e}")
        return None

In [52]:
print(weighted_score(clf, X_test, y_test))

Accuracy: 0.938
Normalized Latency: 0.000 and Latency: 9.423643350601197e-05
Normalized Size: 2554707.000 and Size: 2554707 in Bytes
Final Power-Efficient Score: -255470.044
